In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional, Dropout, BatchNormalization

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers

import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the data into a pandas dataframe
true_df = pd.read_csv("/content/drive/MyDrive/data/True.csv")
fake_df = pd.read_csv("/content/drive/MyDrive/data/Fake.csv")

In [ ]:
t_df = pd.read_csv('/content/drive/MyDrive/data/train2.tsv', sep='\t', names = ['index', 'json', 'class', 'title','topic', 'publisher',
                                                                                          'rep','state','1','2','3','4','5','6',
                                                                                          'person','text'])

In [ ]:
val = t_df = pd.read_csv('/content/drive/MyDrive/data/val2.tsv', sep='\t', names = ['index', 'json', 'class', 'title','topic', 'publisher',
                                                                                          'rep','state','1','2','3','4','5','6',
                                                                                          'person','text'])

In [ ]:
test = t_df = pd.read_csv('/content/drive/MyDrive/data/test2.tsv', sep='\t', names = ['index', 'json', 'class', 'title','topic', 'publisher',
                                                                                          'rep','state','1','2','3','4','5','6',
                                                                                          'person','text'])

##working with dataset 1

In [ ]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake_df.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [ ]:
fake_df['subject'].value_counts()

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

In [ ]:
from logging import exception
unknown_publishers = []
for index, row in enumerate(true_df.text.values):
  try:
    record = row.split(' - ', maxsplit = 1)
    record[1]

    assert(len(record[0])<2120)
  except:
    unknown_publishers.append(index)

In [ ]:
true_df.iloc[unknown_publishers].text

2922    The following statements were posted to the ve...
3488    The White House on Wednesday disclosed a group...
3782    The following statements were posted to the ve...
4358    Neil Gorsuch, President Donald Trump’s appoint...
4465    WASHINGTON The clock began running out this we...
5290    The following statements were posted to the ve...
5588    The following statements were posted to the ve...
5761    The following bullet points are from the U.S. ...
5784    Federal appeals court judge Neil Gorsuch, the ...
6026    The following bullet points are from the U.S. ...
6184    The following bullet points are from the U.S. ...
6660    Republican members of Congress are complaining...
6823    Over the course of the U.S. presidential campa...
7922    After going through a week reminiscent of Napo...
8194    The following timeline charts the origin and s...
8195    Global health officials are racing to better u...
8247    U.S. President Barack Obama visited a street m...
8465    ALGONA

In [ ]:
publisher  = []
tmp_text = []

for index, row in enumerate(true_df.text.values):
  if index in unknown_publishers:
    tmp_text.append(row)
    publisher.append('Unknown')

  else:
    record = row.split('-', maxsplit = 1)
    publisher.append(record[0].strip())
    tmp_text.append(record[1].strip())

In [ ]:
true_df['publisher'] = publisher
true_df['text'] = tmp_text

In [ ]:
true_df.shape

(21417, 5)

In [ ]:
empty_df_f = [index for index, text in enumerate( fake_df.text.tolist()) if str(text).strip()==""]

In [ ]:
fake_df.iloc[empty_df_f]

,title,text,subject,date
10923,TAKE OUR POLL: Who Do You Think President Trum...,,politics,"May 10, 2017"
11041,Joe Scarborough BERATES Mika Brzezinski Over “...,,politics,"Apr 26, 2017"
11190,WATCH TUCKER CARLSON Scorch Sanctuary City May...,,politics,"Apr 6, 2017"
11225,MAYOR OF SANCTUARY CITY: Trump Trying To Make ...,,politics,"Apr 2, 2017"
11236,SHOCKER: Public School Turns Computer Lab Into...,,politics,"Apr 1, 2017"
...,...,...,...,...
21816,BALTIMORE BURNS: MARYLAND GOVERNOR BRINGS IN N...,,left-news,"Apr 27, 2015"
21826,FULL VIDEO: THE BLOCKBUSTER INVESTIGATION INTO...,,left-news,"Apr 25, 2015"
21827,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,left-news,"Apr 25, 2015"
21857,(VIDEO)ICE PROTECTING OBAMA: WON’T RELEASE NAM...,,left-news,"Apr 14, 2015"


In [ ]:
true_df['text'] = true_df['title']+ " "+ true_df['text']
true_df['text'] = true_df['title']+ " "+ true_df['text']

In [ ]:
true_df['text'] = true_df['text'].apply(lambda x: str(x).lower())

In [ ]:
true_df['class'] = 1
fake_df['class'] = 0

In [ ]:
df = true_df.append(fake_df, ignore_index = True)

<ipython-input-23-4a84c9ff6e42>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = true_df.append(fake_df, ignore_index = True)


In [ ]:
df = df.drop(['title','subject','date','publisher'], axis = 1)

In [ ]:
df.shape

(44898, 2)

In [ ]:
df.sample(5)

,text,class
3772,top senate democrat calls on white house to re...,1
22373,"On Saturday, Donald Trump truly became the Pre...",0
23786,Donald Trump may be trying to destroy the fai...,0
41198,The Ecuadorian government admitted on Tuesday ...,0
17712,u.s. still seeking explanation for arrest of s...,1


In [ ]:
df.shape

(44898, 2)

In [ ]:
t_df.head()

,index,json,class,title,topic,publisher,rep,state,1,2,3,4,5,6,person,text
0,0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"Meantime, engineering experts agree the wall w..."
1,1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,She cited layoff notices received by the state...
2,2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"Trump said that McCain ""has done nothing to he..."
3,3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"But spending still goes up. In addition, many ..."
4,4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,Our rating A Democratic Party web video making...


In [ ]:
t_df = t_df.drop(['index','json','title','topic','publisher','rep','state','1','2','3','4','5','6','person'], axis =1)

In [ ]:
val= val.drop(['index','json','title','topic','publisher','rep','state','1','2','3','4','5','6','person'], axis =1)

In [ ]:
test = test.drop(['index','json','title','topic','publisher','rep','state','1','2','3','4','5','6','person'], axis =1)

================================================================================
##Working with dataset 2

In [ ]:
def df_column_switch(df, column1, column2):
    i = list(df.columns)
    a, b = i.index(column1), i.index(column2)
    i[b], i[a] = i[a], i[b]
    df = df[i]
    return df

In [ ]:
true_df = t_df[t_df['class'] == 'true']

In [ ]:
true_df = df_column_switch(true_df, "class", "text")

In [ ]:
fake_df = t_df[t_df['class'] == 'false']

In [ ]:
fake_df = df_column_switch(fake_df, "class", "text")

In [ ]:
true_df['class'] = 1
fake_df['class'] = 0

In [ ]:
df = df.append(fake_df, ignore_index = True)

<ipython-input-38-22317db765f0>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fake_df, ignore_index = True)


In [ ]:
df = df.append(true_df, ignore_index = True)

<ipython-input-39-e7e484e0faa6>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(true_df, ignore_index = True)


In [ ]:
df.shape

(45355, 2)

================================================================================
##working with dataset 3

In [ ]:
true_df = val[val['class'] == 'true']

In [ ]:
true_df = df_column_switch(true_df, "class", "text")

In [ ]:
fake_df = val[val['class'] == 'false']

In [ ]:
fake_df = df_column_switch(fake_df, "class", "text")

In [ ]:
true_df['class'] = 1
fake_df['class'] = 0

In [ ]:
df = df.append(fake_df, ignore_index = True)

<ipython-input-46-22317db765f0>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fake_df, ignore_index = True)


In [ ]:
df = df.append(true_df, ignore_index = True)

<ipython-input-47-e7e484e0faa6>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(true_df, ignore_index = True)


In [ ]:
df.shape

(45787, 2)

================================================================================
##Working with dataset 4




In [ ]:
true_df = test[test['class'] == 'true']

In [ ]:
true_df = df_column_switch(true_df, "class", "text")

In [ ]:
fake_df = test[test['class'] == 'false']

In [ ]:
fake_df = df_column_switch(fake_df, "class", "text")

In [ ]:
true_df['class'] = 1
fake_df['class'] = 0

In [ ]:
df = df.append(fake_df, ignore_index = True)

<ipython-input-54-22317db765f0>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fake_df, ignore_index = True)


In [ ]:
df = df.append(true_df, ignore_index = True)

<ipython-input-55-e7e484e0faa6>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(true_df, ignore_index = True)


================================================================================

In [ ]:
df.shape

(46244, 2)

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-p8a71akq
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-p8a71akq
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7639 sha256=cd05e6b91af15e4fdacf8e794fe4b90e8e8001500cc37ee683928cb878f48446
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_m4hrfu/wheels/d1/c3/bb/559fe93e652b51cbc532f17e9693f3b70055f8560cf06c1fb3
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps

In [ ]:
df.isna().sum()

text     5
class    0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(46239, 2)

In [ ]:
df['text'].apply(lambda x: ps.remove_special_chars(x))

0        as us budget fight looms republicans flip thei...
1        us military to accept transgender recruits on ...
2        senior us republican senator let mr mueller do...
3        fbi russia probe helped by australian diplomat...
4        trump wants postal service to charge much more...
                               ...                        
46239    Romney said repealing the health care law woul...
46240    Obenshains overarching point must be considere...
46241    Gillespie said this is the first time in more ...
46242    Overall in that period Georgias funding throug...
46243    OMalley argued that the cascading effects of c...
Name: text, Length: 46239, dtype: object

In [ ]:

texts = [text for text in df.text]

max_len = 0
texts_len = []
for text in texts:
    texts_len.append(len(text.split()))
    max_len = max(len(text.split()), max_len)

# g = sns.countplot(x=texts_len)
print('Mean length of the texts:', np.mean(texts_len))

Mean length of the texts: 403.2573585068881


In [ ]:

from sklearn.utils import shuffle

# Purify
df = df.iloc[:,[0, -1]]

# Shuffle
df = shuffle(df).reset_index(drop=True)

display(df)

,text,class
0,"Lady Gaga, who protested in front of Trump Tow...",0
1,libyans flee by boat amid 'terrible' violence ...,1
2,"turkey, iraq agree on opening second border ga...",1
3,A video from 1985 was shown last night during ...,0
4,fraud probe to delay liberia election run-off:...,1
...,...,...
46234,You know how Donald Trump stubbornly refuses t...,0
46235,Leona Coakley-Spring has only been running her...,0
46236,The term fake news has become attached to so...,0
46237,Obama decides to nominate Merrick Garland for ...,0


##PreProcess

In [ ]:
# Tokenize the text by splitting it into individual words
df['text'] = df['text'].apply(lambda x: x.split())

In [ ]:
# Remove stopwords from the text
nltk.download('stopwords')
stop_words = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Stem the words in the text
stemmer = PorterStemmer()
df['text'] = df['text'].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
# Lemmatize the words in the text
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Join the words back into a single string
df['text'] = df['text'].apply(lambda x: ' '.join(x))

In [ ]:
df.isna().sum()

text     0
class    0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df['text'].apply(lambda x: ps.remove_special_chars(x))

0        ladi gaga protest front trump tower trump elec...
1        libyan flee boat amid terrible violenc home li...
2        turkey iraq agre open second border gate turki...
3        a video 1985 shown last night dem debat peopl ...
4        fraud probe delay liberia elect runoff elect c...
                               ...                        
46234    you know donald trump stubbornli refus releas ...
46235    leona coakleyspr run consign shop rag riches m...
46236    the term fake news becom attach america major ...
46237    obama decid nomin merrick garland suprem court...
46238    listen zeke emanuel awkward brother chicago ma...
Name: text, Length: 46239, dtype: object

In [ ]:
# Save the preprocessed dataset
df.to_csv('preprocessed_dataset.csv', index=False)

##Fine Tuning BERT with CNN and Attention Mechanism

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads

        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.projection = nn.Linear(hidden_size, hidden_size)

    def forward(self, inputs):
        batch_size = inputs.size(0)

        # Linear transformations
        Q = self.query(inputs).view(batch_size, -1, self.num_heads, self.head_size).transpose(1,2)
        K = self.key(inputs).view(batch_size, -1, self.num_heads, self.head_size).transpose(1,2)
        V = self.value(inputs).view(batch_size, -1, self.num_heads, self.head_size).transpose(1,2)

        # Scaled Dot-Product Attention
        attention_scores = torch.matmul(Q, K.transpose(-1,-2)) / self.head_size**0.5
        attention_probs = F.softmax(attention_scores, dim=-1)
        context = torch.matmul(attention_probs, V).transpose(1,2).contiguous().view(batch_size, -1, self.hidden_size)

        # Output projection
        output = self.projection(context)
        return output

class FakeNewsDetector(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(FakeNewsDetector, self).__init__()
        self.bert = bert_model
        self.attention = MultiHeadAttention(bert_model.config.hidden_size, num_heads=4)
        self.cnn = nn.Conv1d(bert_model.config.hidden_size, bert_model.config.hidden_size, kernel_size=3, padding=1)
        self.classifier = nn.Linear(bert_model.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]  # output shape: (batch_size, seq_length, hidden_size)
        attn_output = self.attention(bert_output)  # output shape: (batch_size, seq_length, hidden_size)
        cnn_output = self.cnn(attn_output.permute(0,2,1)).permute(0,2,1)  # output shape: (batch_size, seq_length, hidden_size)
        cls_output = cnn_output[:, 0, :]  # take the first token as the output
        logits = self.classifier(cls_output)  # output shape: (batch_size, num_classes)
        return logits


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads

        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.projection = nn.Linear(hidden_size, hidden_size)

    def forward(self, inputs):
        batch_size = inputs.size(0)

        # Linear transformations
        Q = self.query(inputs).view(batch_size, -1, self.num_heads, self.head_size).transpose(1,2)
        K = self.key(inputs).view(batch_size, -1, self.num_heads, self.head_size).transpose(1,2)
        V = self.value(inputs).view(batch_size, -1, self.num_heads, self.head_size).transpose(1,2)

        # Scaled Dot-Product Attention
        attention_scores = torch.matmul(Q, K.transpose(-1,-2)) / self.head_size**0.5
        attention_probs = F.softmax(attention_scores, dim=-1)
        context = torch.matmul(attention_probs, V).transpose(1,2).contiguous().view(batch_size, -1, self.hidden_size)

        # Output projection
        output = self.projection(context)
        return output

class SentimentAnalyzer(nn.Module):
    def __init__(self, bert_model, num_sentiments):
        super(SentimentAnalyzer, self).__init__()
        self.bert = bert_model
        self.attention = MultiHeadAttention(bert_model.config.hidden_size, num_heads=4)
        self.cnn = nn.Conv1d(bert_model.config.hidden_size, bert_model.config.hidden_size, kernel_size=3, padding=1)
        self.classifier = nn.Linear(bert_model.config.hidden_size, num_sentiments)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]  # output shape: (batch_size, seq_length, hidden_size)
        attn_output = self.attention(bert_output)  # output shape: (batch_size, seq_length, hidden_size)
        cnn_output = self.cnn(attn_output.permute(0,2,1)).permute(0,2,1)  # output shape: (batch_size, seq_length, hidden_size)
        cls_output = cnn_output[:, 0, :]  # take the first token as the output
        logits = self.classifier(cls_output)  # output shape: (batch_size, num_sentiments)
        return logits



In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel

class BiLSTM_RoBERTa(nn.Module):
    def __init__(self, num_classes, hidden_size=768, num_layers=1, bidirectional=True):
        super(BiLSTM_RoBERTa, self).__init__()
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.bilstm = nn.LSTM(input_size=hidden_size, 
                              hidden_size=hidden_size, 
                              num_layers=num_layers, 
                              batch_first=True, 
                              bidirectional=bidirectional)
        
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=8)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # Pass input through RoBERTa
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]

        # Pass RoBERTa output through BiLSTM
        if self.bidirectional:
            last_hidden_state, _ = self.bilstm(last_hidden_state)
            last_hidden_state = last_hidden_state[:, :, :self.hidden_size] + last_hidden_state[:, :, self.hidden_size:]
        else:
            last_hidden_state, _ = self.bilstm(last_hidden_state)
            
        # Apply self-attention mechanism to the output of BiLSTM and RoBERTa
        att_output, _ = self.attention(last_hidden_state.permute(1,0,2), last_hidden_state.permute(1,0,2), last_hidden_state.permute(1,0,2))
        att_output = att_output.permute(1,0,2)
        
        # Pass the final output through a linear layer
        logits = self.fc(att_output[:, -1, :])
        #return logits
        return torch.softmax(logits, dim=-1)


In [ ]:
# Load the preprocessed dataset
df = pd.read_csv('preprocessed_dataset.csv')

In [ ]:
df.isna().sum()

text     630
class      0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df['text'].apply(lambda x: ps.remove_special_chars(x))

0        ladi gaga protest front trump tower trump elec...
1        libyan flee boat amid terrible violenc home li...
2        turkey iraq agre open second border gate turki...
3        a video 1985 shown last night dem debat peopl ...
4        fraud probe delay liberia elect runoff elect c...
                               ...                        
46234    you know donald trump stubbornli refus releas ...
46235    leona coakleyspr run consign shop rag riches m...
46236    the term fake news becom attach america major ...
46237    obama decid nomin merrick garland suprem court...
46238    listen zeke emanuel awkward brother chicago ma...
Name: text, Length: 45609, dtype: object

In [ ]:
# Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['class'], test_size=0.2, random_state=42)

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Tokenize the train and test sets
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [ ]:
# Convert the train and test labels to tensors
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

In [ ]:
# Create a PyTorch DataLoader for the train and test sets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
fake_news_detector = FakeNewsDetector(bert_model, num_classes=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fake_news_detector.parameters(), lr=1e-5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fake_news_detector.to(device)

FakeNewsDetector(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
num_epochs = 4
for epoch in range(num_epochs):
  # Training loop
  for batch in train_loader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)
    optimizer.zero_grad()

    outputs = fake_news_detector(input_ids, attention_mask)

    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()

In [ ]:
# Test loop
with torch.no_grad():
    correct = 0
    total = 0

    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = fake_news_detector(input_ids, attention_mask)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Epoch: {}, Test Accuracy: {}%'.format(epoch+1, accuracy))

In [ ]:
fake_news_detector.save_pretrained("MODEL.hdf5")

In [ ]:
torch.save(fake_news_detector.state_dict(), '/path/to/save/finetuned_bert_weights.pt')


Make sure to replace 'fake_news_detector_weights.pt' with the path to the file containing the pre-trained weights for the FakeNewsDetector model.

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from fake_news_detector import FakeNewsDetector

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Instantiate FakeNewsDetector model
num_classes = 2
model = FakeNewsDetector(bert_model, num_classes)

# Load pre-trained weights
model.load_state_dict(torch.load('fake_news_detector_weights.pt'))

# Example input text
input_text = "Researchers have discovered a new species of dinosaur in Argentina."

# Tokenize input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True)
attention_mask = [1] * len(input_ids)

# Convert to tensors
input_ids = torch.tensor(input_ids).unsqueeze(0)  # shape: (batch_size=1, seq_length)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)  # shape: (batch_size=1, seq_length)

# Pass input through model
model.eval()
with torch.no_grad():
    logits = model(input_ids, attention_mask)

# Get predicted class
probs = torch.softmax(logits, dim=-1)
pred_class = torch.argmax(probs, dim=-1).item()

# Print results
print(f"Input text: {input_text}")
print(f"Predicted class: {pred_class}")
